In [2]:
import os
import sys
import random
import warnings
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from tqdm import tqdm
from itertools import chain

from keras.utils import to_categorical
from keras.callbacks import Callback

from keras import layers
from keras import optimizers
from keras.models import Model, load_model
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers import Concatenate, multiply
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Conv1D
from keras.layers import ZeroPadding1D
from keras.layers import GlobalAveragePooling1D
from keras.layers.pooling import MaxPooling1D

from keras.layers import Deconv2D,LeakyReLU,add,Reshape
from keras import losses

from keras import regularizers
from keras.layers import Conv2D

from keras.losses import categorical_crossentropy

from keras.layers.core import Lambda

from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

from keras.models import model_from_json

from keras import  backend as K

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
session = tf.Session(config=config)

KTF.set_session(session )

In [3]:
train_batch = 128
test_batch = 128

train_num = 40020
test_num = 13108

cross = "cross_1"

snr_str = "-6_db"

In [4]:
def generate_train_data():
    
    train_data = np.zeros((train_batch, 2048, 1), dtype = np.float32)
    d_train_lab = np.zeros((train_batch, 2048, 1), dtype = np.float32)
    c_train_lab = np.zeros((train_batch))
    
    flag = 0
    
    while True:
        
        list = random.sample(range(train_num), train_num)
        
        for id in list:
            
            num_id = str(id)
            
            np_train_data = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/train_data/" + num_id + "_train.npy")
            npd_train_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/no_db/train_data/" + num_id + "_train.npy") 
            npc_train_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/train_lab/" + num_id + "_lab.npy")
            
            train_data[flag, :, :] = np_train_data
            d_train_lab[flag, :, :] = npd_train_lab    
            c_train_lab[flag] = npc_train_lab
            
            flag += 1
            
            if flag >= train_batch:
                
                train_hot_lab = to_categorical(c_train_lab, num_classes=10)
                
                yield [train_data], [d_train_lab]
                
                flag = 0
                train_data = np.zeros((train_batch, 2048, 1), dtype = np.float32)
                d_train_lab = np.zeros((train_batch, 2048, 1), dtype = np.float32)
                c_train_lab = np.zeros((train_batch))

        
def generate_test_data():
    
    test_data = np.zeros((test_batch, 2048, 1), dtype = np.float32)
    d_test_lab = np.zeros((test_batch, 2048, 1), dtype = np.float32)
    c_test_lab = np.zeros((test_batch))
    
    flag = 0
    
    while True:
        
        list = random.sample(range(test_num), test_num)
        
        for id in list:
            
            num_id = str(id)
            
            np_test_data = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_data/" + num_id + "_test.npy")
            npd_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/no_db/test_data/" + num_id + "_test.npy") 
            npc_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_lab/" + num_id + "_lab.npy")
            
            test_data[flag, :, :] = np_test_data
            d_test_lab[flag, :, :] = npd_test_lab    
            c_test_lab[flag] = npc_test_lab
            
            flag += 1
            
            if flag >= test_batch:
                
                test_hot_lab = to_categorical(c_test_lab, num_classes=10)
                
                yield [test_data], [d_test_lab]
                
                flag = 0
                test_data = np.zeros((test_batch, 2048, 1), dtype = np.float32)
                d_test_lab = np.zeros((test_batch, 2048, 1), dtype = np.float32)
                c_test_lab = np.zeros((test_batch))

In [5]:
def get_all_test_data():
    
    test_data = np.zeros((test_num, 2048, 1), dtype = np.float32)
    d_test_lab = np.zeros((test_num, 2048, 1), dtype = np.float32)
    c_test_lab = np.zeros((test_num))
    
    flag = 0
    list = random.sample(range(test_num), test_num)
    
    for id in list:
        
        num_id = str(id)
        
        np_test_data = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_data/" + num_id + "_test.npy")
        npd_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/no_db/test_data/" + num_id + "_test.npy") 
        npc_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_lab/" + num_id + "_lab.npy")
            
        test_data[flag, :, :] = np_test_data
        d_test_lab[flag, :, :] = npd_test_lab    
        c_test_lab[flag] = npc_test_lab
        
        flag += 1
        
    test_hot_lab = to_categorical(c_test_lab, num_classes=10)
    
    return test_data, d_test_lab, test_hot_lab

In [6]:

slope = 0.3

def Denosing_CNN2():
    
    input = Input(shape=[2048, 1])

    D1 = Conv1D(filters=16, kernel_size=12, strides=4, padding="same")(input)
    D1 = LeakyReLU(alpha=slope)(D1)
        
    D2 = Conv1D(filters=32, kernel_size=6, strides=4, padding="same")(D1)
    D2 = LeakyReLU(alpha=slope)(D2)    
    
    D3 = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(D2)
    D3 = LeakyReLU(alpha=slope)(D3)
    
    D4 = Conv1D(filters=128, kernel_size=3, strides=2, padding="same")(D3)
    D4 = LeakyReLU(alpha=slope)(D4)    

    
    Dx_4 = Reshape(target_shape=[32, 1, 128])(D4)    
    
    
    Dx_5 = Deconv2D(filters=128, kernel_size=[3, 1], strides=[2, 1], padding="same")(Dx_4)
    Dx_5 = LeakyReLU(alpha=slope)(Dx_5)   
    
    Dx_6 = Deconv2D(filters=64, kernel_size=[3, 1], strides=[2, 1], padding="same")(Dx_5)
    Dx_6 = LeakyReLU(alpha=slope)(Dx_6)
    
    Dx_7 = Deconv2D(filters=32, kernel_size=[6, 1], strides=[4, 1], padding="same")(Dx_6)
    Dx_7 = LeakyReLU(alpha=slope)(Dx_7)

    Dx_8 = Deconv2D(filters=16, kernel_size=[12, 1], strides=[4, 1], padding="same")(Dx_7)
    Dx_8 = LeakyReLU(alpha=slope)(Dx_8)
    
    Dx_9 = Deconv2D(filters=1, kernel_size=[1, 1], strides=[1, 1], padding="same")(Dx_8)
    Dx_9 = LeakyReLU(alpha=slope)(Dx_9)    
    
    Dx_10 = Reshape(target_shape=[2048, 1], name='out1')(Dx_9)      

         
    Total_Model = Model(inputs=input, outputs=Dx_10)  
    
    return Total_Model

In [7]:
adam = optimizers.adam(lr = 0.0001)

model = Denosing_CNN2()

model.compile(optimizer = 'adam', loss='mean_squared_error', metrics = ['mean_squared_error'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 16)           208       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 32)           3104      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128, 32)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 64, 64)            6208      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 64, 64)            0         
__________

In [ ]:
class MyCbk(Callback):

    def __init__(self, model):
        self.model_to_save = model
            
    def on_epoch_end(self, epoch, logs=None):
        
        print('save model----model_at_epoch_%d.h5' % epoch)
        self.model_to_save.save('model/2/model_%d.h5' % epoch)
        
cbk = MyCbk(model)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.90, patience=5, verbose=0, mode='auto', cooldown=0, min_lr=0.0000001)

results = model.fit_generator(generate_train_data(), steps_per_epoch = train_num/train_batch, epochs = 100, validation_data = generate_test_data(), validation_steps = test_num/test_batch, verbose=1, callbacks=[cbk, reduceLR])

In [ ]:
all_test_data, all_d_lab, all_hot_lab = get_all_test_data()

def SNR_score(noise_data, pure_data):
    
    noise_data = np.array(noise_data)
    pure_data = np.array(pure_data)
    
    noise_power = np.sum((noise_data-pure_data)**2)/len(noise_data)
    pure_power = np.sum(pure_data**2)/len(pure_data)
    
    SNR_d = 10*math.log10(pure_power/noise_power)
    
    return SNR_d


def MSE_score(noise_data, pure_data):
    
    noise_data = np.array(noise_data)
    pure_data = np.array(pure_data)
    
    MSE_d = np.sum((noise_data-pure_data)**2)/(len(noise_data)*2048)
    
    return MSE_d 


SNR_list = np.zeros((100))
MSE_list = np.zeros((100))

In [ ]:
for model_id in range(99,100):
    
    str_id = str(model_id)
    model =  load_model('model/2/model_'+ str_id +'.h5') 
    
    range_num = int(test_num/test_batch)
    
    input_d_lab = []
    output_d_lab = []
    
    for num_id in range(range_num):
        
        test_data = all_test_data[num_id*test_batch:test_batch+num_id*test_batch,:,:]
        d_lab = all_d_lab[num_id*test_batch:test_batch+num_id*test_batch,:,:]
    
        pre_d = model.predict(test_data, batch_size=test_batch, verbose=0)

        input_d_lab.extend(d_lab)
        output_d_lab.extend(pre_d)
        
    SNR = SNR_score(output_d_lab, input_d_lab)
    MSE = MSE_score(output_d_lab, input_d_lab)

    SNR_list[model_id] = SNR
    MSE_list[model_id] = MSE
    
    print("第"+ str(model_id) + "个模型的结果：")
    print(SNR)
    print(MSE)
    
    print("------------------------------------")
    
    K.clear_session()
    tf.reset_default_graph()


In [ ]:
SNR_str = "["
MSE_str = "["

for id in range(80, 100):
    
    SNR_num = round(SNR_list[id], 5)
    MSE_num = round(MSE_list[id], 5)

    SNR_str = SNR_str + str(SNR_num)
    MSE_str = MSE_str + str(MSE_num)
    
    if id == 99:
        SNR_str = SNR_str + "]"
        MSE_str = MSE_str + "]"
    else:
        SNR_str = SNR_str + ", "
        MSE_str = MSE_str + ", "

print("SNR_score:")
print(SNR_str)
print("MSE_score:")
print(MSE_str)

print('SNR: '+str(SNR_list[80:99].max()))
print('MSE: '+str(MSE_list[80:99].min()))
print("**------------**-------------**---------------**")